In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hii'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['python'], 'python'), (['what', 'is', 'python'], 'python'), (['Is', 'Python', 'front', 'end', 'or', 'backend', '?'], 'pythonqu'), (['What', 'is', 'the', 'main', 'use', 'of', 'Python', '?'], 'pythonq'), (['use', 'of', 'python'], 'pythonq'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 

In [3]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

52 documents
12 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'python', 'pythonq', 'pythonqu', 'thanks']
95 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'backend', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'end', 'entry', 'find', 'for', 'front', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hii', 'history', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'main', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'or', 'patient', 'pharmacy', 'pressure', 'provide', 'python', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'transfe

In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [5]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

Training data created


<ipython-input-5-b01d127cc81c>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [6]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Users\sharad gabel\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [7]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
11/11 [==============================] - 1s 3ms/step - loss: 2.5073 - accuracy: 0.0577
Epoch 2/200
11/11 [==============================] - 0s 2ms/step - loss: 2.3773 - accuracy: 0.1538
Epoch 3/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2879 - accuracy: 0.2692
Epoch 4/200
11/11 [==============================] - 0s 2ms/step - loss: 2.1824 - accuracy: 0.2885
Epoch 5/200
11/11 [==============================] - 0s 2ms/step - loss: 2.1298 - accuracy: 0.2115
Epoch 6/200
11/11 [==============================] - 0s 2ms/step - loss: 2.0034 - accuracy: 0.2885
Epoch 7/200
11/11 [==============================] - 0s 3ms/step - loss: 1.7730 - accuracy: 0.4615
Epoch 8/200
11/11 [==============================] - 0s 2ms/step - loss: 1.7201 - accuracy: 0.4231
Epoch 9/200
11/11 [==============================] - 0s 3ms/step - loss: 1.6917 - accuracy: 0.5385
Epoch 10/200
11/11 [==============================] - 0s 3ms/step - loss: 1.4518 - accuracy: 0.5577
Epoch 11/

11/11 [==============================] - 0s 3ms/step - loss: 0.0318 - accuracy: 1.0000
Epoch 84/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0756 - accuracy: 0.9808
Epoch 85/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1133 - accuracy: 0.9808
Epoch 86/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0609 - accuracy: 1.0000
Epoch 87/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1408 - accuracy: 0.9615
Epoch 88/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1207 - accuracy: 0.9808
Epoch 89/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0492 - accuracy: 1.0000
Epoch 90/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0595 - accuracy: 1.0000
Epoch 91/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0886 - accuracy: 0.9808
Epoch 92/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0594 - accuracy: 1.0000
Epoch 93/200


Epoch 164/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0162 - accuracy: 1.0000
Epoch 165/200
11/11 [==============================] - 0s 5ms/step - loss: 0.1120 - accuracy: 0.9615
Epoch 166/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0325 - accuracy: 0.9808
Epoch 167/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0301 - accuracy: 1.0000
Epoch 168/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0166 - accuracy: 1.0000
Epoch 169/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0292 - accuracy: 0.9808
Epoch 170/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0432 - accuracy: 0.9808
Epoch 171/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0560 - accuracy: 0.9808
Epoch 172/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 173/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0042 - accurac